## Pseudo-labing

The goal of this python file is to get a dataset that is a merge of both the two datasets that we have and use it to train our model.

We have:
* Dataset A → images + age + gender + race
* Dataset B → images + emotion only

We want to:
1. Train a demographics model on Dataset A
2. Use it to predict age/gender/race on Dataset B
3. Save those predictions as pseudo-labels with confidence
4. Merge the datasets safely

This is called **pseudo-labeling**

Important: these are not true labels, so we store them separately and track confidence.


In [1]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import cv2

/Users/renubandaru/Code/GitHub/facial-profiler/deeplearning/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### STEP 1 -Load Dataset A and Dataset B

In [2]:
# Load the datasets
dfA = pd.read_csv("utk_face_labels.csv")   # age, gender, race
dfB = pd.read_csv("raf_labels.csv")   # emotion only

print(dfA.head())
print(dfB.head())

                                          image_path  age  gender  Race
0  source_data/UTK-Face/part3/27_0_1_201701201338...   27       0     1
1  source_data/UTK-Face/part3/24_0_3_201701191655...   24       0     3
2  source_data/UTK-Face/part3/8_1_0_2017011715460...    8       1     0
3  source_data/UTK-Face/part3/85_1_0_201701202226...   85       1     0
4  source_data/UTK-Face/part3/26_1_0_201701191929...   26       1     0
                                          image_path  emotion
0  source_data/raf/DATASET/train/7/train_11651_al...        7
1  source_data/raf/DATASET/train/7/train_10043_al...        7
2  source_data/raf/DATASET/train/7/train_11301_al...        7
3  source_data/raf/DATASET/train/7/train_10513_al...        7
4  source_data/raf/DATASET/train/7/train_11148_al...        7


In [3]:
#STEP 1.5 — Validate and Clean Image Paths

def validate_image_paths(df):
    """Remove rows with corrupt or unreadable images."""
    valid_paths = []
    for path in df['image_path']:
        try:
            img = cv2.imread(path)
            if img is not None and img.size > 0:
                valid_paths.append(path)
        except:
            continue
    return df[df['image_path'].isin(valid_paths)].reset_index(drop=True)

print(f"Original dfA shape: {dfA.shape}")
dfA = validate_image_paths(dfA)
print(f"Cleaned dfA shape: {dfA.shape}")

Original dfA shape: (24102, 4)


Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: bad Huffman code


Cleaned dfA shape: (24102, 4)


### STEP 2 — Convert Age to Bins

We are doing this step because the exact age prediction is noisy and difficult. Age classificiation into bins is more stable.

In [4]:
# Create age bins
age_bins = [0,10,20,30,40,50,60,200]

def age_to_bin(age):
    return np.digitize(age, age_bins) - 1

# Convert age to bins and add as a new column in dfA
dfA["age_bin"] = dfA["age"].apply(age_to_bin)

# printing the unique age bins to verify
print("Unique age bins in dfA:", dfA["age_bin"].unique())

print(dfA[["age", "age_bin"]].head(10))


Unique age bins in dfA: [2 0 6 5 3 4 1]
   age  age_bin
0   27        2
1   24        2
2    8        0
3   85        6
4   26        2
5   57        5
6   33        3
7   78        6
8   45        4
9   34        3


### STEP 3 — Converting Gender to Integers

Neural networks need numeric labels. Therefore we need to convert them accordingly. 

In [5]:
# Drop rows where gender class is 3 since its meaning is unclear and it may be an outlier or error in the dataset
print(f"\nOriginal dataset shape: {dfA.shape}")
dfA = dfA[dfA['gender'] != 3]
print(f"Dataset shape after dropping gender=3: {dfA.shape}")

# Convert gender to categorical if it's not already numeric
if dfA["gender"].dtype == "object":
    dfA["gender"] = dfA["gender"].astype("category").cat.codes

num_age = dfA["age_bin"].nunique()
num_gender = dfA["gender"].nunique()

print(f"Number of age bins: {num_age}")
print(f"Number of gender classes: {num_gender}")


Original dataset shape: (24102, 5)
Dataset shape after dropping gender=3: (24102, 5)
Number of age bins: 7
Number of gender classes: 2


### STEP 4 — Train/Validation Split

Both shuffle=True and stratify=dfA["age_bin"] are important for proper model training and evaluation. 

* Benefits of shuffle=True:
    - Prevents order bias: Without shuffling, if your data is sorted by age, the model might learn patterns based on the order rather than actual features
    - Better generalization: Random mixing ensures the model sees diverse examples in each batch
    - Prevents overfitting to data patterns: Shuffling breaks any inherent ordering that might exist in your dataset

* Benefits of stratify=dfA["age_bin"]:
    - Balanced age distribution: Ensures both training and validation sets have the same proportion of each age group
    - Prevents bias: Without stratification, some age groups might be underrepresented in validation, leading to unreliable performance metrics
    - More accurate evaluation: Your validation set will better represent the real-world age distribution
    - Stable training: Prevents scenarios where certain age groups are only in training or only in validation

In [6]:
# Train/validation split for dataset A with stratification on age bins
trainA, valA = train_test_split(
    dfA,
    test_size=0.2,
    random_state=42,
    shuffle=True,
    stratify=dfA["age_bin"])

 ### STEP 5 — Create TensorFlow Data Pipelin


In [7]:
IMG_SIZE = 224  # Define a consistent image size for the model
BATCH_SIZE = 32

def preprocess_image(path):
    try:
        img = tf.io.read_file(path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0
        return img
    except:
        # Return blank/gray image on error
        return tf.zeros((IMG_SIZE, IMG_SIZE, 3))

def create_datasetA(df, training=True):
    image_paths = df["image_path"].values
    age = df["age_bin"].values
    gender = df["gender"].values
    
    # Create a TensorFlow dataset from the image paths and labels
    ds = tf.data.Dataset.from_tensor_slices((image_paths, age, gender))

    # Map the dataset to load and preprocess images and return labels
    def load_data(path, age, gender):
        img = preprocess_image(path)
        # Apply stronger augmentation only during training to improve generalization
        if training:
            img = tf.image.random_flip_left_right(img)
            img = tf.image.random_crop(img, size=[IMG_SIZE, IMG_SIZE, 3])
            img = tf.image.random_brightness(img, max_delta=0.1)
            img = tf.image.random_contrast(img, lower=0.9, upper=1.1)
            img = tf.image.random_saturation(img, lower=0.9, upper=1.1)
        return img, {"age": age, "gender": gender}
    
    # Map the dataset to load and preprocess images and return labels
    ds = ds.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

# Create datasets for training and validation
train_ds = create_datasetA(trainA, training=True)
val_ds = create_datasetA(valA, training=False)

### STEP 6 — Build Multi-Output Model

This is a very important step

We use:
- Pretrained MobileNetV2
- 2 output heads:
    - age
    - gender
Shared feature extractor → multiple tasks

In [8]:
# Build the model - MobileNetV2 as the base model for feature extraction
base_model = keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights="imagenet",
)

base_model.trainable = False  # Freeze the base model first

# Add custom layers on top of the base model for age and gender
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)

# Separate processing for age head (more capacity)
x_age = layers.Dense(512, activation='relu')(x)
x_age = layers.Dropout(0.4)(x_age)
age_output = layers.Dense(num_age, activation='softmax', name='age')(x_age)

# Gender head (keep simple)
gender_output = layers.Dense(num_gender, activation='softmax', name='gender')(x)

# Output layers for age and gender
age_output = layers.Dense(num_age, activation="softmax", kernel_regularizer=keras.regularizers.l2(0.001), name="age")(x) # adding L2 regularization to prevent overfitting
gender_output = layers.Dense(num_gender, activation="softmax", name="gender")(x)


# Create the model with two outputs
model = keras.Model(inputs=inputs, outputs=[age_output, gender_output])


### STEP 7 — Compile Model

In [9]:
import tensorflow.keras.backend as K

def focal_loss(gamma=2., alpha=.25):
    def loss(y_true, y_pred):
        y_true = K.cast(y_true, 'int32')
        y_true = K.one_hot(y_true, K.shape(y_pred)[-1])
        p_t = K.sum(y_true * y_pred, axis=-1)
        loss = -alpha * K.pow(1 - p_t, gamma) * K.log(K.clip(p_t, 1e-7, 1.0))
        return loss
    return loss

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss={
        "age": focal_loss(gamma=2., alpha=.25), # focal loss for imbalanced age bins

        "gender": keras.losses.SparseCategoricalCrossentropy()
    },
    loss_weights={"age": 2.0, "gender": 1.0},  # ← Emphasize age
    metrics={
        "age": "accuracy",
        "gender": "accuracy"
    }
)

### STEP 8 — Train on Dataset A

**Why Two-Phase (Frozen → Fine-tune) is Better Than Direct Fine-tuning**

The Problem with Starting Unfrozen:
* MobileNetV2 pretrained on ImageNet already has good generic features (edges, textures, shapes)
* If you unfreeze immediately with high learning rate, you overwrite these good features with poor updates (since your face dataset is small initially)
* Result: Catastrophic forgetting — model loses useful pretrained knowledge

**Why Two-Phase Works:**
1. Phase 1 (Frozen, 5 epochs):
    - Backbone weights locked → can't change
    - Only age/gender heads learn to adapt pretrained features to your task
    - Stable & fast — reuses pretrained knowledge effectively

2. Phase 2 (Unfrozen, 15 epochs, LR=1e-5):

    - After the age head has learned, unfreeze last 20 backbone layers
    - Use a very low learning rate (1e-5) → slow, small updates only
    - Gently fine-tune backbone features to specialize for age/gender
    - Backbone is already initialized with good features, so small updates improve accuracy

Analogy:
* Frozen phase: Transfer pre-trained knowledge to your task (quick win)
* Fine-tune phase: Customize pretrained knowledge (gradual improvement)

In [10]:
# Training: two-phase process with callbacks and fine-tuning
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# Callbacks: reduce LR on plateau and early stopping to prevent wasting time
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-7, verbose=1)
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)


# 1) Initial training with frozen base (base model was frozen earlier)
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=[reduce_lr, early_stop]
)

# 2) Unfreeze the base model for fine-tuning: unfreeze last convolutional blocks
base_model.trainable = True
# Freeze all layers up to a specific point to avoid catastrophic forgetting
fine_tune_at = max(0, len(base_model.layers) - 20)  # unfreeze last ~20 layers (adjust as needed)
for i, layer in enumerate(base_model.layers):
    layer.trainable = (i >= fine_tune_at)

# Re-compile with a much lower learning rate for fine-tuning
model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss={"age": keras.losses.SparseCategoricalCrossentropy(), "gender": keras.losses.SparseCategoricalCrossentropy()},
    loss_weights={"age": 2.0, "gender": 1.0},
    metrics={"age": "accuracy", "gender": "accuracy"}
)

# Continue training (fine-tuning)
history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[reduce_lr, early_stop]
)

Epoch 1/5
 99/603 ━━━━━━━━━━━━━━━━━━━━ 3:19 396ms/step - age_accuracy: 0.1460 - age_loss: 0.4633 - gender_accuracy: 0.3830 - gender_loss: 1.0149 - loss: 1.9555

2026-02-19 15:15:54.571434: W tensorflow/core/lib/png/png_io.cc:95] PNG warning: iCCP: known incorrect sRGB profile


137/603 ━━━━━━━━━━━━━━━━━━━━ 3:06 400ms/step - age_accuracy: 0.1475 - age_loss: 0.4583 - gender_accuracy: 0.3846 - gender_loss: 1.0070 - loss: 1.9375

Corrupt JPEG data: premature end of data segment


586/603 ━━━━━━━━━━━━━━━━━━━━ 6s 406ms/step - age_accuracy: 0.1797 - age_loss: 0.4159 - gender_accuracy: 0.4176 - gender_loss: 0.9370 - loss: 1.7826

...

137/603 ━━━━━━━━━━━━━━━━━━━━ 3:49 492ms/step - age_accuracy: 0.7529 - age_loss: 0.7231 - gender_accuracy: 0.9229 - gender_loss: 0.2011 - loss: 1.6617

Corrupt JPEG data: premature end of data segment


586/603 ━━━━━━━━━━━━━━━━━━━━ 8s 477ms/step - age_accuracy: 0.7497 - age_loss: 0.7170 - gender_accuracy: 0.9213 - gender_loss: 0.2054 - loss: 1.6537

Corrupt JPEG data: premature end of data segment


603/603 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step - age_accuracy: 0.7497 - age_loss: 0.7168 - gender_accuracy: 0.9214 - gender_loss: 0.2053 - loss: 1.6534
Epoch 15: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
603/603 ━━━━━━━━━━━━━━━━━━━━ 387s 642ms/step - age_accuracy: 0.7497 - age_loss: 0.7168 - gender_accuracy: 0.9214 - gender_loss: 0.2053 - loss: 1.6534 - val_age_accuracy: 0.5652 - val_age_loss: 1.0800 - val_gender_accuracy: 0.8944 - val_gender_loss: 0.2623 - val_loss: 2.4366 - learning_rate: 1.0000e-05


In [11]:
# STEP 8.5 — Evaluate Age Performance Per Bin
from sklearn.metrics import classification_report

# Get predictions on validation set
val_preds = model.predict(val_ds)
age_preds = val_preds[0].argmax(axis=1)  # age predictions
age_true = valA['age_bin'].values

print("Age Classification Report:")
print(classification_report(age_true, age_preds, target_names=[f'Bin {i}' for i in range(num_age)]))

151/151 ━━━━━━━━━━━━━━━━━━━━ 58s 378ms/step
Age Classification Report:
              precision    recall  f1-score   support

       Bin 0       0.85      0.86      0.86       667
       Bin 1       0.59      0.40      0.47       311
       Bin 2       0.60      0.76      0.67      1473
       Bin 3       0.38      0.27      0.32       909
       Bin 4       0.28      0.22      0.25       450
       Bin 5       0.38      0.39      0.38       462
       Bin 6       0.66      0.72      0.69       549

    accuracy                           0.57      4821
   macro avg       0.53      0.52      0.52      4821
weighted avg       0.55      0.57      0.55      4821



### STEP 9 — Predict Pseudo-Labels for Dataset B

Create loader for B

In [12]:
# Generate pseudo-labels for dataset B using the trained model

IMG_SIZE = 224  # must match model input size

# Function to load and preprocess images for dataset B
def load_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    
    img = keras.applications.mobilenet_v2.preprocess_input(img)
    
    return img

# Create a TensorFlow dataset for dataset B
def create_datasetB(df):
    image_paths = df["image_path"].values
    
    # Create a TensorFlow dataset from the image paths
    ds = tf.data.Dataset.from_tensor_slices((image_paths))

    # Map the dataset to load and preprocess images
    def process(path) :
        img = load_image(path)
        return img, path
    
    # Map the dataset to load and preprocess images and return labels
    ds = ds.map(process, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE) # Prefetch for performance
    return ds


dsB = create_datasetB(dfB)

In [13]:
# Generate pseudo-labels predictions for dataset B using the trained model

# Store predictions and confidence scores
age_preds = []
gender_preds = []

age_conf = []
gender_conf = []

# Iterate through dataset B and get predictions from the model
for images, paths in dsB:
    age_p, gender_p = model.predict(images, verbose=0)
    
    age_preds.extend(np.argmax(age_p, axis=1))
    gender_preds.extend(np.argmax(gender_p, axis=1))
    
    age_conf.extend(np.max(age_p, axis=1))
    gender_conf.extend(np.max(gender_p, axis=1))


2026-02-19 16:32:53.184117: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### STEP 10 — Create Augmented Dataset B

In [14]:
# Add the pseudo-labels and confidence scores to the original dataframe for dataset B
dfB["age_pseudo"] = age_preds
dfB["gender_pseudo"] = gender_preds

dfB["age_conf"] = age_conf
dfB["gender_conf"] = gender_conf

print(dfB.head())

                                          image_path  emotion  age_pseudo  \
0  source_data/raf/DATASET/train/7/train_11651_al...        7           2   
1  source_data/raf/DATASET/train/7/train_10043_al...        7           0   
2  source_data/raf/DATASET/train/7/train_11301_al...        7           2   
3  source_data/raf/DATASET/train/7/train_10513_al...        7           2   
4  source_data/raf/DATASET/train/7/train_11148_al...        7           2   

   gender_pseudo  age_conf  gender_conf  
0              0  0.360979     0.501508  
1              1  0.671543     0.961124  
2              1  0.919729     0.892669  
3              1  0.489154     0.758168  
4              0  0.736868     0.571540  


### STEP 11 — Confidence Filtering

We don’t trust low-confidence predictions. Therefore it is necessary for us to drop those predictions

* -1 = unknown
* Others = pseudo-label

In [15]:
THRESHOLD = 0.50 # Set a confidence threshold for accepting pseudo-labels

# Set pseudo-labels to -1 for samples where confidence is below the threshold
dfB.loc[dfB["age_conf"] < THRESHOLD, "age_pseudo"] = -1
dfB.loc[dfB["gender_conf"] < THRESHOLD, "gender_pseudo"] = -1

# Save the updated dataframe with pseudo-labels to a new CSV file`
dfB.to_csv("B_with_pseudo_labels.csv", index=False)

### STEP 12 — Merge Datasets

Keep true vs pseudo separate!

In [16]:
print(dfA.columns)

Index(['image_path', 'age', 'gender', 'Race', 'age_bin'], dtype='object')


In [17]:
# Merge datasets A and B using the true labels from dataset A and the pseudo-labels from dataset B.

dfA["emotion"] = -1  # no emotion label in A

dfA["age"] = dfA["age_bin"]        # true age
dfA["gender"] = dfA["gender"]  # true gender

dfA = dfA[["image_path", "age", "gender", "emotion"]]


# Since dataset B does not have true labels, we will use the pseudo-labels as the "true" labels for merging. 
# We will also keep the original columns for clarity, but they will be filled with NaN since we don't have true labels for dataset B.
dfB["age"] = dfB["age_pseudo"]
dfB["gender"] = dfB["gender_pseudo"]
dfB["emotion"] = dfB["emotion"]

dfB = dfB[["image_path", "age", "gender", "emotion"]]

# Merge the two datasets
merged = pd.concat([dfA, dfB], ignore_index=True)
merged.to_csv("merged_dataset.csv", index=False)
